In [2]:
#Cellule 1 — Imports
#But : importer les librairies nécessaires (NumPy pour les calculs, random pour le hasard).

import numpy as np
import random


In [3]:
#Cellule 2 — Classe de l’environnement simple

#But : créer l’atelier simplifié (pas encore version Gym), dans lequel les stocks évoluent selon les actions.
# Cette cellule définit l’atelier : 
#comment les stocks changent selon l’action et comment l’environnement avance d’un pas.

class SimpleWorkshopEnv:
    def __init__(self, max_steps=50):
        self.max_steps = max_steps
        self.reset()

    def reset(self):
        # Stock initial aléatoire entre 0 et 10
        self.stock_raw = np.random.randint(0, 11)

        # Toujours 0 produit fini au départ
        self.stock_sell = 0

        self.steps = 0
        return (self.stock_raw, self.stock_sell)


    def step(self, action):
        reward = 0

        if action == 1:  # Produire Produit 1
            if self.stock_raw >= 1:
                self.stock_raw -= 1
                self.stock_sell += 1
                reward = 2

        elif action == 2:  # Produire Produit 2
            if self.stock_raw >= 2:
                self.stock_raw -= 2
                self.stock_sell += 1
                reward = 20

        elif action == 3:  # Commander du stock
            self.stock_raw += 5
            reward = -5

        elif action == 0:  # Attendre
        # Si on n'a plus de stock, attendre n'a aucun sens → petite pénalité
            reward = -1 if self.stock_raw == 0 else 0


        # Bornes pour éviter des valeurs aberrantes
        self.stock_raw = max(0, min(10, self.stock_raw))
        self.stock_sell = max(0, min(10, self.stock_sell))

        # Incrémentation du temps
        self.steps += 1
        done = self.steps >= self.max_steps

        next_state = (self.stock_raw, self.stock_sell)
        return next_state, reward, done, {}


In [4]:
#Cellule 3 — Instanciation de l’environnement

#But : créer une instance de l’atelier pour pouvoir interagir avec lui.
#On crée l’environnement dans lequel l’agent va apprendre.

env = SimpleWorkshopEnv(max_steps=50)


In [5]:
#Cellule 4 — Définition des tailles et création de la table-Q

#But : créer la table-Q vide et définir les hyperparamètres du Q-learning.

# Dimensions de l'espace d'états
n_states_raw = 11      # 0 à 10
n_states_sell = 11     # 0 à 10
n_actions = 4          # 0,1,2,3

# Table-Q initialisée à zéro
Q = np.zeros((n_states_raw, n_states_sell, n_actions))

# Hyperparamètres du Q-learning
alpha = 0.1     # taux d'apprentissage
gamma = 0.95    # importance du futur

# Paramètres d'exploration
epsilon = 1.0         # exploration maximale au début
epsilon_min = 0.1
epsilon_decay = 0.995


In [6]:
#Cellule 5 — Fonction de choix d’action (ε-greedy)

#But : choisir action aléatoire ou action optimale selon Q-table.
#Cette cellule définit comment l’agent choisit ses actions : parfois au hasard, parfois selon la table-Q.

def choose_action(state, epsilon):
    stock_raw, stock_sell = state

    # Exploration
    if random.random() < epsilon:
        return random.randint(0, n_actions - 1)

    # Exploitation
    return int(np.argmax(Q[stock_raw, stock_sell, :]))


In [7]:
#Cellule 6 — Boucle d’apprentissage du Q-learning

#But : exécuter plusieurs épisodes, mettre à jour la Q-table avec la formule de Bellman.
#Cette cellule entraîne l’agent : il joue, reçoit des récompenses, met à jour la table-Q et apprend une stratégie.
n_episodes = 5000
rewards_per_episode = []

for episode in range(n_episodes):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        # 1. Choisir une action
        action = choose_action(state, epsilon)

        # 2. Exécuter l'action
        next_state, reward, done, info = env.step(action)

        # 3. Indices pour la Q-table
        sr, ss = state
        nsr, nss = next_state

        # 4. Q-learning update (formule de Bellman)
        best_next_Q = np.max(Q[nsr, nss, :])
        Q[sr, ss, action] += alpha * (reward + gamma * best_next_Q - Q[sr, ss, action])

        # 5. Passer à l'état suivant
        state = next_state
        total_reward += reward

    rewards_per_episode.append(total_reward)

    # 6. Décroissance de epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    # Petit affichage
    if (episode + 1) % 50 == 0:
        print(f"Episode {episode+1} : Reward total = {total_reward:.1f}, epsilon = {epsilon:.3f}")


Episode 50 : Reward total = 273.0, epsilon = 0.778
Episode 100 : Reward total = 364.0, epsilon = 0.606
Episode 150 : Reward total = 389.0, epsilon = 0.471
Episode 200 : Reward total = 431.0, epsilon = 0.367
Episode 250 : Reward total = 488.0, epsilon = 0.286
Episode 300 : Reward total = 516.0, epsilon = 0.222
Episode 350 : Reward total = 542.0, epsilon = 0.173
Episode 400 : Reward total = 556.0, epsilon = 0.135
Episode 450 : Reward total = 553.0, epsilon = 0.105
Episode 500 : Reward total = 573.0, epsilon = 0.100
Episode 550 : Reward total = 556.0, epsilon = 0.100
Episode 600 : Reward total = 601.0, epsilon = 0.100
Episode 650 : Reward total = 612.0, epsilon = 0.100
Episode 700 : Reward total = 621.0, epsilon = 0.100
Episode 750 : Reward total = 615.0, epsilon = 0.100
Episode 800 : Reward total = 544.0, epsilon = 0.100
Episode 850 : Reward total = 650.0, epsilon = 0.100
Episode 900 : Reward total = 559.0, epsilon = 0.100
Episode 950 : Reward total = 560.0, epsilon = 0.100
Episode 1000 

In [17]:
#Cellule 7 — Inspection de la politique apprise

#But : afficher les meilleures actions apprises pour différents états.
#Cette cellule permet de voir ce que l’agent a réellement appris comme stratégie.

def best_action_for_state(stock_raw, stock_sell):
    q_vals = Q[stock_raw, stock_sell, :]
    best_a = int(np.argmax(q_vals))
    return best_a, q_vals

for sr in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    action, qv = best_action_for_state(sr, 0)
    print(f"État ({sr},0) → meilleure action = {action}, Q-values = {qv}")


État (0,0) → meilleure action = 3, Q-values = [131.72178299 161.24922249 138.44395731 237.77559485]
État (1,0) → meilleure action = 3, Q-values = [126.92561584 108.25180142 123.99816928 239.94820801]
État (2,0) → meilleure action = 2, Q-values = [142.7412958   31.54307784 253.76827934  88.62314424]
État (3,0) → meilleure action = 3, Q-values = [177.25273938 165.16177712  86.57443537 255.18234946]
État (4,0) → meilleure action = 2, Q-values = [141.13037211 195.8282422  241.62520086 159.1246792 ]
État (5,0) → meilleure action = 2, Q-values = [180.42561544 227.1083797  255.55325773 173.41608801]
État (6,0) → meilleure action = 2, Q-values = [222.54250593 232.62560199 257.84021896 236.57852096]
État (7,0) → meilleure action = 2, Q-values = [224.57983038 115.20080539 271.07187299 150.94347478]
État (8,0) → meilleure action = 2, Q-values = [220.76102325 198.9719982  273.87615733 224.03195213]
État (9,0) → meilleure action = 2, Q-values = [101.90398826 113.18671373 254.60079761 214.74870957]
